# **Imports**

In [1]:
import os
os.chdir("../")
os.chdir(r"src")

import warnings
warnings.filterwarnings("ignore")

from Processing.preprocess_parliament import *

os.chdir("../")

In [3]:
#df_par = read_HouseOfCommons(keep_date=True, rd_lines=False, size=100)
df_DM = pd.read_csv('data/raw_corpuses/articlesDM.csv', index_col=[0])
df_DE = pd.read_csv('data/raw_corpuses/articlesDE.csv', index_col=[0])
df_GUA = pd.read_csv('data/raw_corpuses/articlesGUA.csv', index_col=[0])
df_MET = pd.read_csv('data/raw_corpuses/articlesMET.csv', index_col=[0])
df_TE = pd.read_csv('data/raw_corpuses/articlesTE.csv', index_col=[0])

# **Dates**

We create here a function that finds the date of publication from some newspapers' urls, as we didn't get it by the webscraping

In [4]:
def find_date_url(url:str):
    for i in range(14):
        if str(2010+i) in url :
            return str(2010+i)

In [5]:
df_MET['date'] = df_MET['url'].apply(find_date_url)
df_GUA['date'] = df_GUA['url'].apply(find_date_url)
df_TE['date'] = df_TE['url'].apply(find_date_url)

In [6]:
def cut_date(date):
    try :
        return(date[:4])
    except :
        return None

In [7]:
df_DM['date'] = df_DM['date'].apply(cut_date)
df_DE['date'] = df_DE['date'].apply(cut_date)

# **Parliament database filtering**

In [7]:
df_par['date'] = df_par['date'].apply(cut_date)
df_par = df_par.rename(columns={'date':'year'})
df_par['source'] = 'par'

In [8]:
list_stem_topics = process_list_BigTech_words(topics)

In [9]:
df_par = keep_Bigtech_speeches(df_par, list_stem_topics)

/Users/alexandrequeant/Desktop/Stage Toulouse /Stage_TSE_2023/Stage_TSE_2023/src/preprocess_parliament.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['agenda', 'lines_to_keep'], inplace=True)


In [10]:
df_par.reset_index(inplace=True)
df_par.to_csv('df_par.csv', index=False)

In [11]:
len(df_par)

238082

# **Columns**

We format all the dataframes to have the same relevant columns

In [8]:
df_DM = df_DM[['date', 'fulltext', 'headline', 'url']]
df_DM = df_DM.rename(columns={"fulltext": "text", "date": "year", "headline":"agenda"})
df_DM['source'] = 'DM'

In [9]:
df_DE = df_DE[['date', 'fulltext', 'headline', 'url']]
df_DE = df_DE.rename(columns={"fulltext": "text", "date": "year", "headline":"agenda"})
df_DE['source'] = 'DE'

In [10]:
df_GUA = df_GUA[['date', 'fulltext', 'summary', 'url']]
df_GUA = df_GUA.rename(columns={"fulltext": "text", "date": "year", "summary":"agenda"})
df_GUA['source'] = 'GUA'

In [11]:
df_MET = df_MET[['date', 'fulltext', 'headline', 'url']]
df_MET= df_MET.rename(columns={"fulltext": "text", "date": "year", "headline":"agenda"})
df_MET['source'] = 'MET'

In [12]:
df_TE = df_TE[['date', 'fulltext', 'headline', 'url']]
df_TE= df_TE.rename(columns={"fulltext": "text", "date": "year", "headline":"agenda"})
df_TE['source'] = 'TE'

## **Retrofiltering**

We study here the urls of our articles in order to save for which keywords they have been selected during the webscraping phase. We can then use it in order to filter articles by keywords during our study. 

In [13]:
key_words = ['apple', 'iphone', 'ipad', 'mac', 'watch', 'macbook', 'ios', 'airpods', 'app-store', 'itunes', 'icloud', 'beats', 'siri', 'shazam', 'cook', 'jobs', 'meta', 'facebook', 'messenger', 'instagram', 'oculus', 'whatsApp', 'zuckerberg', 'olivan', 'clegg', 'social-media', 'amazon', 'prime', 'whole-foods', 'zappos', 'pillpack', 'twitch', 'audible', 'goodreads', 'imdb', 'bezos', 'jassy', 'olsavsky', 'alexander', 'google', 'android', 'chrome', 'gmail', 'maps', 'playstore', 'pixel', 'waze', 'youtube', 'alphabet', 'mandiant', 'fitbit', 'looker', 'nest', 'doubleclick', 'page', 'brin', 'pichai', 'kurian', 'microsoft', 'windows', 'cortana', 'excel', 'explorer', 'edge', 'teams', 'outlook', 'powerpoint', 'skype', 'surface', 'xbox', 'linkedIn', 'github', 'mojang', 'gates', 'nadella', 'gafa', 'gafam', 'big-tech', 'consumer-privacy', 'tech']

In [ ]:
def retro_filt_speech(speech):
    l = []
    for word in key_words:
        if word in str(speech.lower()):
            l.append(word)
    return l

df_par['keywords'] = df_par['text'].apply(retro_filt_speech)

In [15]:
def retro_filt_url(url):
    l = []
    for word in key_words:
        if word in str(url):
            l.append(word)
    return l

df_DE['keywords'] = df_DE['url'].apply(retro_filt_url)
df_DM['keywords'] = df_DM['url'].apply(retro_filt_url)
df_GUA['keywords'] = df_GUA['url'].apply(retro_filt_url)
df_TE['keywords'] = df_TE['url'].apply(retro_filt_url)
df_MET['keywords'] = df_MET['url'].apply(retro_filt_url)

We have here a loop to attibute a relevant date to articles that still miss one

In [16]:
for t in range(24):
    for i in df_TE.index:
        if df_TE['year'][i] == None :
                try :
                    df_TE['year'][i] = df_TE['year'][i+1]
                except :
                    i = i

In [17]:
df_TE = df_TE.dropna()

In [18]:
df = pd.concat([df_GUA, df_DE, df_DM, df_MET, df_TE])

In [28]:
df = pd.concat([df_par, df_GUA, df_DE, df_DM, df_MET, df_TE])

In [19]:
len(df)

59842

Our intermediate dataframe appears here 

# **Final cleaning**

We then clean the dataframe and save it

In [20]:
# On cleen les titres des speechs
df['agenda'] = df['agenda'].apply(clean, args=('unigram',))
#On nettoie le texte des speeches 
df['text'] = df['text'].apply(clean, args=('unigram',))

In [22]:
os.getcwd()

'/Users/alexandrequeant/Desktop/Travail-TSE'

In [25]:
df.to_csv('data/without parliament/FinalDataframes/FinalDataFrame_WP.csv', index=False)

In [26]:
len(df)

59842